<a href="https://colab.research.google.com/github/pbbdydx/MLB/blob/main/fine_tuning_llms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

import os
import zipfile
import numpy as np
from PIL import Image
import tensorflow as tf
from sklearn.model_selection import train_test_split


--2024-11-06 21:53:42--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.117.207, 173.194.202.207, 173.194.203.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.117.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   124MB/s    in 0.5s    

2024-11-06 21:53:43 (124 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [5]:
# Unzip dataset
local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

# Define paths
base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Load images and labels
x_data = []
y_data = []
image_size = (150, 150)  # Resize all images to 150x150

# Load cat images
cats_dir = os.path.join(train_dir, "cats")
for img_name in os.listdir(cats_dir):
    img_path = os.path.join(cats_dir, img_name)
    img = Image.open(img_path).convert("RGB").resize(image_size)
    x_data.append(np.array(img))
    y_data.append(1)  # Label for cats

# Load dog images
dogs_dir = os.path.join(train_dir, "dogs")
for img_name in os.listdir(dogs_dir):
    img_path = os.path.join(dogs_dir, img_name)
    img = Image.open(img_path).convert("RGB").resize(image_size)
    x_data.append(np.array(img))
    y_data.append(0)  # Label for dogs

# Convert to numpy arrays
x_data = np.array(x_data)
y_data = np.array(y_data)

# Split into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

# Normalize pixel values
x_train = x_train / 255.0
x_test = x_test / 255.0

# Load pre-trained ResNet50 model
pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(150, 150, 3),
    pooling='avg',
    weights='imagenet'
)

# Freeze the base model
pretrained_model.trainable = False

# Add custom classification layers
model = tf.keras.Sequential([
    pretrained_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification: 1 for cat, 0 for dog
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

# Fine-tuning: unfreeze some layers and re-train with a lower learning rate
pretrained_model.trainable = True
for layer in pretrained_model.layers[:-10]:  # Freeze earlier layers to avoid overfitting
    layer.trainable = False

# Compile with a lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Fine-tune the model
fine_tune_history = model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 184s 4s/step - accuracy: 0.5164 - loss: 0.7742 - val_accuracy: 0.5425 - val_loss: 0.6858
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 187s 3s/step - accuracy: 0.5371 - loss: 0.6968 - val_accuracy: 0.5325 - val_loss: 0.6919
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 157s 3s/step - accuracy: 0.5433 - loss: 0.6877 - val_accuracy: 0.5700 - val_loss: 0.6738
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 167s 3s/step - accuracy: 0.5428 - loss: 0.6861 - val_accuracy: 0.5600 - val_loss: 0.6745
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 193s 3s/step - accuracy: 0.5823 - loss: 0.6736 - val_accuracy: 0.5450 - val_loss: 0.6768
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 168s 3s/step - accuracy: 0.5764 - loss: 0.6730 - val_accuracy: 0.5875 - val_loss: 0.6723
Epoch 7/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 201s 3s/step - accuracy: 0.6112 - loss: 0.6652 - val_accuracy: 0.5725 - val_loss: 0.6726
Epoch 8/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 192s 3s/step - accurac